In [18]:
###### INCLUSION CRITERIA ANALYSIS - BAG-OF-WORDS FEATURE EXTRACTION###
####The below code 
#Creators: Xiaoru Dong
#Contributors: Linh Hoang
#Last updated: 04/8/2019
#The below Python script is used to generate features and to create the Weka input files 
#corresponding to the bag-of-word feature extraction and selection approach that we implemented in the study

In [19]:
from collections import OrderedDict
import pandas as pd
import os

######GLOBAL FUNCTIONS TO EXTRACT FEATURE AND TO GENERATE WEKA INPUT FILES####

#Set a function for outputing features
def writefeatures(path, features):
    with open(path,"w") as fp: 
        for word in features:
            fp.write(word + "\n")
            
#Set a function to get an arff file used as the input in Weka
def writewekainput(path, features, lineNum, eachline, rct):
    with open(path,"w") as fp:        #this line of code to create a new file
        fp.write('''@RELATION classification_analysis\n''')
    
        for i in range(len(features)): 
            fp.write('''@ATTRIBUTE ''' + "word" + str(i) + ''' NUMERIC\n''') 
    
        fp.write('''@ATTRIBUTE class {1.0, 0.0}\n''')
        fp.write('''@DATA\n''')

        #check if a word appear in the feature list or not. If yes, then write "1", if no then write "0"
        for i in range(lineNum): 
            binary = {}
            for word in features:
                if word in eachline[i]:
                    binary[word] = 1
                else:
                    binary[word] = 0
                fp.write(str(binary[word]) + ",")
            fp.write(rct[i] + "\n")


In [20]:
#####BAG-OF-WORDS FEATURE EXTRACTION#####
###Input: Inclusion_Criteria_Annotation.csv
###Output: AllWords_weka_input.arff

            
#Get the path of the folder that contains the annotation file
#Change this path to the actual path of your local machine
path = "/Users/phuong/Downloads/Inclusion_Criteria_Revised/"

#Read the inclusion criteria annotation file
file = path + 'Inclusion_Criteria_Annotation_New.csv'
df = pd.read_csv(file, encoding = "ISO-8859-1")
lineNum = len(df)

#The list of stop words
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
              "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
              'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 
              'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'if', 'because', 'as', 'until', 'of', 'at', 'by', 'for', 'with', 'about', 'between', 'into',
              'through', 'during', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
              'under', 'again', 'further', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'few', 'more', 
              'most', 'some', 'such', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 
              'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain']

# Extract bag-of-words features
allwords_features = []
wordlist_oneline = []
rct = []
wordlist_alllines = []

# split the sentence into words and save in a list
for i in range(lineNum): 
    
    string_rct = str(df['Only RCTs'][i])
    if string_rct == 'nan':
        string_rct = '0.0'
    if string_rct == 'x':
        string_rct = '1.0'
    rct.append(string_rct)
    
    string = str(df['Inclusion Criteria'][i]).lower().strip()
    wordlist_oneline = string.split(' ')
    
    for j in range(len(wordlist_oneline)):
        wordlist_oneline[j] = wordlist_oneline[j].replace('.', '')
        wordlist_oneline[j] = wordlist_oneline[j].replace(')', '')
        wordlist_oneline[j] = wordlist_oneline[j].replace('(', '')
        wordlist_oneline[j] = wordlist_oneline[j].replace(',', '')
        wordlist_oneline[j] = wordlist_oneline[j].replace("'", '')
        wordlist_oneline[j] = wordlist_oneline[j].replace('"', '')
        allwords_features.append(wordlist_oneline[j])
    wordlist_alllines.append(wordlist_oneline)
    
#remove empty element in the list
allwords_features = list(OrderedDict.fromkeys(allwords_features))
allwords_features = [x for x in allwords_features if x] 

# remove stop words
allwords_features = [word for word in allwords_features if word not in stop_words]

# output features and save in a csv file
allwords_features_savepath = path + "AllWords.csv" 
writefeatures(allwords_features_savepath, allwords_features)

# output the weka input file
wekainput_allwords_path= path + "AllWords_weka_input.arff"
writewekainput(wekainput_allwords_path, allwords_features, lineNum, wordlist_alllines, rct)

# We use the weka to read the"AllWords_weka_input.arff" input file. Then, we run the machine learning model in
# weka to get the results precision, recall and F-meaure on 3 algorithms: Random Forest, J48 and Naïve Bayes. 

#####END OF BAG-OF-WORD FEATURE EXTRACTION#####